### Quick Description
This notebook downloads every video subtitle listed in **download_descriptions.csv**, and creates a single output file with all sentences concatenated

In [ ]:
import pandas as pd
import subprocess
import logging
import os

from youtube_transcript_api import YouTubeTranscriptApi
from tqdm import tqdm

In [ ]:
download_desc_file = "../data/download_descriptions.csv"

df = pd.read_csv(download_desc_file)
for i, row in tqdm(df.iterrows(), total=len(df)):
    video_id_and_channel = row["url"].split("v=")[1]
    video_id = video_id_and_channel.split("&")[0]

    try:
        if row["has-en-sub"]:
            transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
            subtitles = transcript_list.find_manually_created_transcript(["en"]).fetch()
        else:
            transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
            subtitles = transcript_list.find_generated_transcript(["en"]).fetch()

        subtitles = [sentence["text"] for sentence in subtitles]
        subtitles = " ".join(subtitles)

        with open(os.path.join("../data/00_raw/", video_id), 'w') as f:
            f.write(subtitles)
    except:
        logging.error(f"Error while downloading the {i}-th video.")